In [1]:
"""
Task-1: Create and Insert Student Document
Objective:
• Construct a new document representing a student with specific fields.
• Persist the document in the baudb database, students collection.
"""
#I created the data in mongodb and used it from there.
from pymongo import MongoClient 

client = MongoClient('mongodb://localhost:27017/')
db = client['baudb']
collection = db['students']

for student in collection.find():
    print(student)


{'_id': '91237649292', 'name': 'Ben Linus', 'birth_year': 2009, 'courses': [{'_id': 'ain1001', 'name': 'Computer Programming I (Pyhton)', 'grade': 'A-', 'lecturer': {'_id': ' 26416956694', 'name': 'Jack Shephard'}, 'schedule': {'_id': 'schedule101', 'start_time': '2023-11-28T10:00:00Z', 'end_time': '2023-11-28T12:00:00Z', 'room': 'A101'}, 'credit': 3.67}, {'_id': 'ain1002', 'name': 'Computer Programming II (C++)', 'grade': 'B+', 'lecturer': {'_id': '94424438570', 'name': 'Kate Austen'}, 'schedule': {'_id': 'schedule202', 'start_time': '2023-11-29T13:00:00Z', 'end_time': '2023-11-29T15:00:00Z', 'room': 'B202'}, 'credit': 3.33}, {'_id': 'ain3005', 'name': 'Advanced Pyhton', 'grade': 'C-', 'lecturer': {'_id': '17289807236', 'name': 'John Sanjuan'}, 'schedule': {'_id': 'schedule305', 'start_time': '2023-11-30T16:00:00Z', 'end_time': '2023-11-30T19:00:00Z', 'room': 'DSC01'}, 'credit': 1.67}, {'_id': 'ain2001', 'name': 'Data Science', 'grade': 'D+', 'lecturer': {'_id': ' 3467890123', 'name':

In [2]:
"""
Task-2: Find Students with Specific Grade in Course
Retrieve all students enrolled in "ain1001" (Computer Programming I) during 2023 with a grade of C or higher. 
"""

grades = ["A", "A-", "B+", "B", "C+", "C", "C-", "D+", "D", "F","I"]

students = collection.find({
    "courses": {
        "$elemMatch": {
            "_id": "ain1001",        
            "grade": {"$in": grades},  
            "schedule.start_time": {"$gte": "2023-01-01T00:00:00Z", "$lt": "2024-01-01T00:00:00Z"}  
        }
    }
})

for student in students:
    print("Name:", student['name'])
    for course in student['courses']:
        if course['_id'] == "ain1001" and course['grade'] in grades:
            print(f"  Courses: {course['name']}")
            print(f"  Grade: {course['grade']}")
            print(f"  Start Time: {course['schedule']['start_time']}")
            print(f"  End Time: {course['schedule']['end_time']}")
    print()

Name: Ben Linus
  Courses: Computer Programming I (Pyhton)
  Grade: A-
  Start Time: 2023-11-28T10:00:00Z
  End Time: 2023-11-28T12:00:00Z



In [3]:
""" 
Task-3: Add Credit Field based on Grade
Introduce a new field named "credit" within each student's course object.
"""
grade_to_credit = {
    "A": 4,
    "A-": 3.67,
    "B+": 3.33,
    "B": 3,
    "B-": 2.67,
    "C+": 2.33,
    "C": 2,
    "C-": 1.67,
    "D+": 1.33,
    "D": 1,
    "I": 0,
    "F": 0
}

students = collection.find()

for student in students:
    for course in student.get("courses", []): 
        grade = course.get("grade", "F") 
        credit = grade_to_credit.get(grade, 0)  
        course["credit"] = credit  
        print(f" - Student: {student['name']}, Course: {course['name']}, Grade: {grade}, Credit: {credit}, Lecturer: {course['lecturer']}")
    
    collection.update_one(
        {"_id": student["_id"]},  
        {"$set": {"courses": student["courses"]}} 
    )

print("Credit field added and updates logged successfully.")

 - Student: Ben Linus, Course: Computer Programming I (Pyhton), Grade: A-, Credit: 3.67, Lecturer: {'_id': ' 26416956694', 'name': 'Jack Shephard'}
 - Student: Ben Linus, Course: Computer Programming II (C++), Grade: B+, Credit: 3.33, Lecturer: {'_id': '94424438570', 'name': 'Kate Austen'}
 - Student: Ben Linus, Course: Advanced Pyhton, Grade: C-, Credit: 1.67, Lecturer: {'_id': '17289807236', 'name': 'John Sanjuan'}
 - Student: Ben Linus, Course: Data Science, Grade: D+, Credit: 1.33, Lecturer: {'_id': ' 3467890123', 'name': 'Emily Larson'}
 - Student: Ben Linus, Course: Database Systems and Cloud Computing, Grade: B, Credit: 3, Lecturer: {'_id': '7320652441', 'name': 'Oscar Brown'}
Credit field added and updates logged successfully.


In [4]:
"""
Task-4: Delete Students with Low Average Credit
Delete a student whose average credit is less than 2.0 from the database.
"""
student_grades = [
    {
        "_id": "1",
        "name": "Ben Linus",
        "courses": [
            {"name": "Computer Programming I (Python)", "grade": "A-", "credit": 3.67, "lecturer": {"_id": "26416956694", "name": "Jack Shephard"}},
            {"name": "Computer Programming II (C++)", "grade": "B+", "credit": 3.33, "lecturer": {"_id": "94424438570", "name": "Kate Austen"}},
            {"name": "Advanced Python", "grade": "C-", "credit": 1.67, "lecturer": {"_id": "17289807236", "name": "John Sanjuan"}},
            {"name": "Data Science", "grade": "D+", "credit": 1.33, "lecturer": {"_id": "3467890123", "name": "Emily Larson"}},
            {"name": "Database Systems and Cloud Computing", "grade": "B", "credit": 3, "lecturer": {"_id": "7320652441", "name": "Oscar Brown"}}
        ]
    }
]

class AvgCreditCalculator:
    def delete_one(self, query):
        print(f"Simulating deletion for student with _id: {query['_id']}")

collection = AvgCreditCalculator()

low_credit_students_exist = False

for student in student_grades:
    total_credit = 0
    total_courses = 0
    for course in student.get("courses", []):
        total_credit += course.get("credit", 0)
        total_courses += 1
    
    if total_courses > 0:
        average_credit = total_credit / total_courses
        print(f"Student: {student['name']}, Average Credit: {average_credit:.2f}")

        if average_credit < 2.0:
            print(f"Deleting student: {student['name']}, Average Credit: {average_credit:.2f}")
            collection.delete_one({"_id": student["_id"]})
            low_credit_students_exist = True

if not low_credit_students_exist:
    print("There are no students with less than 2.0 credits.")

Student: Ben Linus, Average Credit: 2.60
There are no students with less than 2.0 credits.
